# Setup (colab only)

In [ ]:
!git clone https://github.com/chris838/alpha-zero-general.git

In [ ]:
%cd 'alpha-zero-general'

In [ ]:
!pip install -r docker/requirements.txt

# Train AlphaZero

In [1]:
import logging
import coloredlogs
from Coach import Coach
from utils import dotdict
from santorini.keras.NNet import NNetWrapper
from santorini.SantoriniGame import SantoriniGame

In [2]:
import Arena
from MCTS import MCTS

from santorini.SantoriniPlayers import (
    RandomPlayer,
    HumanSantoriniPlayer,
    GreedySantoriniPlayer,
)

import numpy as np
from utils import *

In [3]:
log = logging.getLogger(__name__)
coloredlogs.install(level='INFO')  # Change this to DEBUG to see more info.

In [4]:
args = dotdict({
    'numIters': 1000,
    'numEps': 100,              # Number of complete self-play games to simulate during a new iteration.
    'tempThreshold': 15,        #
    'updateThreshold': 0.6,     # During arena playoff, new neural net will be accepted if threshold or more of games are won.
    'maxlenOfQueue': 200000,    # Number of game examples to train the neural networks.
    'numMCTSSims': 25,          # Number of games moves for MCTS to simulate.
    'arenaCompare': 40,         # Number of games to play during arena play to determine if new net will be accepted.
    'cpuct': 1,
    'checkpoint': './temp/',
    'load_model': False,
    'numItersForTrainExamplesHistory': 20,
})

In [5]:
# If you have a pre-trained model, you can load it here.
import os
if os.path.exists(os.path.join('pretrained_models', 'santorini', 'keras', '5x5', 'best.pth.tar.index')):
  print ("Using best pre-existing model")
  args['load_model'] = True
  args['load_folder_file'] = ('pretrained_models/santorini/keras/5x5','best.pth.tar')
else:
  print ("Not using best pre-existing model")

Not using best pre-existing model


In [7]:
game = SantoriniGame(5)

In [8]:
nnet = NNetWrapper(game)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-07-16 18:16:38.684521: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-16 18:16:38.684703: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
if args.load_model:
    print('Loading checkpoint "{}/{}"...'.format(args.load_folder_file[0], args.load_folder_file[1]))
    nnet.load_checkpoint(args.load_folder_file[0], args.load_folder_file[1])
else:
    print('Not loading a checkpoint.')

Not loading a checkpoint.


In [10]:
# Set very low iterations to let this notebook run in its entirety.

# In reality, training a model, even as simple as the one for Dots and Boxes, can take several hours or days.
args['numIters'] = 1000
args['numEps'] = 100
args['arenaCompare'] = 40

In [11]:
coach = Coach(game, nnet, args)

In [12]:
%time coach.learn()

2022-07-16 18:16:42 chris-pro.local Coach[45616] INFO Starting Iter #1 ...
Self Play:   0%|          | 0/100 [00:00<?, ?it/s]2022-07-16 18:16:42.423996: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-16 18:16:42.489868: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
Self Play: 100%|██████████| 100/100 [38:57<00:00, 23.38s/it]


Checkpoint Directory exists! 
Epoch 1/10


2022-07-16 18:55:45.204651: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


396/396 [==============================] - 16s 39ms/step - loss: 6.1134 - pi_loss: 4.9333 - v_loss: 1.1802
Epoch 2/10
396/396 [==============================] - 16s 39ms/step - loss: 5.9114 - pi_loss: 4.8276 - v_loss: 1.0838
Epoch 3/10
396/396 [==============================] - 16s 39ms/step - loss: 5.7603 - pi_loss: 4.7591 - v_loss: 1.0012
Epoch 4/10
396/396 [==============================] - 16s 39ms/step - loss: 5.5503 - pi_loss: 4.7215 - v_loss: 0.8289
Epoch 5/10
396/396 [==============================] - 15s 39ms/step - loss: 5.3226 - pi_loss: 4.6906 - v_loss: 0.6320
Epoch 6/10
396/396 [==============================] - 16s 40ms/step - loss: 5.1035 - pi_loss: 4.6489 - v_loss: 0.4545
Epoch 7/10
396/396 [==============================] - 16s 39ms/step - loss: 4.9203 - pi_loss: 4.5908 - v_loss: 0.3295
Epoch 8/10
396/396 [==============================] - 15s 39ms/step - loss: 4.7533 - pi_loss: 4.5060 - v_loss: 0.2473
Epoch 9/10
396/396 [==============================] - 16s 39ms/step

2022-07-16 18:58:21 chris-pro.local Coach[45616] INFO PITTING AGAINST PREVIOUS VERSION
Arena.playGames (2): 100%|██████████| 20/20 [08:36<00:00, 25.81s/it]
2022-07-16 19:15:33 chris-pro.local Coach[45616] INFO NEW/PREV WINS : 17 / 23 ; DRAWS : 0
2022-07-16 19:15:33 chris-pro.local Coach[45616] INFO REJECTING NEW MODEL
2022-07-16 19:15:33 chris-pro.local Coach[45616] INFO Starting Iter #2 ...
Self Play: 100%|██████████| 100/100 [38:39<00:00, 23.20s/it]


...

Checkpoint Directory exists! 
Epoch 1/10
1105/1105 [==============================] - 44s 39ms/step - loss: 4.9051 - pi_loss: 4.4137 - v_loss: 0.4914
Epoch 2/10
1105/1105 [==============================] - 43s 39ms/step - loss: 4.6744 - pi_loss: 4.3346 - v_loss: 0.3399
Epoch 3/10
1105/1105 [==============================] - 44s 39ms/step - loss: 4.4867 - pi_loss: 4.2281 - v_loss: 0.2586
Epoch 4/10
1105/1105 [==============================] - 44s 39ms/step - loss: 4.2819 - pi_loss: 4.0710 - v_loss: 0.2109
Epoch 5/10
1105/1105 [==============================] - 44s 39ms/step - loss: 4.0242 - pi_loss: 3.8434 - v_loss: 0.1808
Epoch 6/10
1105/1105 [==============================] - 43s 39ms/step - loss: 3.7049 - pi_loss: 3.5418 - v_loss: 0.1632
Epoch 7/10
1105/1105 [==============================] - 44s 39ms/step - loss: 3.3504 - pi_loss: 3.1963 - v_loss: 0.1542
Epoch 8/10
1105/1105 [==============================] - 43s 39ms/step - loss: 2.9835 - pi_loss: 2.8432 - v_loss: 0.1403
Epoch 9/10

2022-07-16 20:53:48 chris-pro.local Coach[45616] INFO PITTING AGAINST PREVIOUS VERSION
Arena.playGames (2):  10%|█         | 2/20 [00:43<06:37, 22.10s/it]2022-07-16 21:01:59 chris-pro.local MCTS[45616] ERROR All valid moves were masked, doing a workaround.
2022-07-16 21:01:59 chris-pro.local MCTS[45616] ERROR All valid moves were masked, doing a workaround.
Arena.playGames (2):  30%|███       | 6/20 [02:17<05:15, 22.52s/it]2022-07-16 21:03:36 chris-pro.local MCTS[45616] ERROR All valid moves were masked, doing a workaround.
2022-07-16 21:03:36 chris-pro.local MCTS[45616] ERROR All valid moves were masked, doing a workaround.
2022-07-16 21:03:37 chris-pro.local MCTS[45616] ERROR All valid moves were masked, doing a workaround.
2022-07-16 21:03:37 chris-pro.local MCTS[45616] ERROR All valid moves were masked, doing a workaround.
2022-07-16 21:03:37 chris-pro.local MCTS[45616] ERROR All valid moves were masked, doing a workaround.
2022-07-16 21:03:37 chris-pro.local MCTS[45616] ERROR All 

Checkpoint Directory exists! 
Epoch 1/10
1438/1438 [==============================] - 57s 40ms/step - loss: 4.9338 - pi_loss: 4.3261 - v_loss: 0.6077
Epoch 2/10
1438/1438 [==============================] - 57s 39ms/step - loss: 4.7172 - pi_loss: 4.2631 - v_loss: 0.4540
Epoch 3/10
1438/1438 [==============================] - 57s 40ms/step - loss: 4.5439 - pi_loss: 4.1903 - v_loss: 0.3535
Epoch 4/10
1438/1438 [==============================] - 57s 40ms/step - loss: 4.3763 - pi_loss: 4.0875 - v_loss: 0.2888
Epoch 5/10
1438/1438 [==============================] - 58s 40ms/step - loss: 4.1689 - pi_loss: 3.9332 - v_loss: 0.2357
Epoch 6/10
1438/1438 [==============================] - 57s 40ms/step - loss: 3.9149 - pi_loss: 3.7092 - v_loss: 0.2057
Epoch 7/10
1438/1438 [==============================] - 57s 40ms/step - loss: 3.5993 - pi_loss: 3.4150 - v_loss: 0.1843
Epoch 8/10
1438/1438 [==============================] - 57s 39ms/step - loss: 3.2388 - pi_loss: 3.0692 - v_loss: 0.1697
Epoch 9/10

2022-07-16 21:50:41 chris-pro.local Coach[45616] INFO PITTING AGAINST PREVIOUS VERSION
Arena.playGames (2):  15%|█▌        | 3/20 [01:17<07:25, 26.23s/it]2022-07-16 21:59:05 chris-pro.local MCTS[45616] ERROR All valid moves were masked, doing a workaround.
2022-07-16 21:59:11 chris-pro.local MCTS[45616] ERROR All valid moves were masked, doing a workaround.
Arena.playGames (2): 100%|██████████| 20/20 [07:01<00:00, 21.05s/it]
2022-07-16 22:04:25 chris-pro.local Coach[45616] INFO NEW/PREV WINS : 16 / 24 ; DRAWS : 0
2022-07-16 22:04:25 chris-pro.local Coach[45616] INFO REJECTING NEW MODEL
2022-07-16 22:04:25 chris-pro.local Coach[45616] INFO Starting Iter #5 ...
Self Play: 100%|██████████| 100/100 [35:16<00:00, 21.17s/it]


Checkpoint Directory exists! 
Epoch 1/10
1798/1798 [==============================] - 71s 39ms/step - loss: 4.9468 - pi_loss: 4.2636 - v_loss: 0.6832
Epoch 2/10
1798/1798 [==============================] - 71s 39ms/step - loss: 4.7198 - pi_loss: 4.1936 - v_loss: 0.5262
Epoch 3/10
1798/1798 [==============================] - 71s 39ms/step - loss: 4.5229 - pi_loss: 4.1161 - v_loss: 0.4068
Epoch 4/10
1798/1798 [==============================] - 71s 39ms/step - loss: 4.3221 - pi_loss: 3.9999 - v_loss: 0.3222
Epoch 5/10
1798/1798 [==============================] - 71s 39ms/step - loss: 4.0954 - pi_loss: 3.8323 - v_loss: 0.2631
Epoch 6/10
1798/1798 [==============================] - 71s 39ms/step - loss: 3.8176 - pi_loss: 3.5922 - v_loss: 0.2254
Epoch 7/10
1721/1798 [===========================>..] - ETA: 28:09 - loss: 3.4791 - pi_loss: 3.2792 - v_loss: 0.1999

In [ ]:
%matplotlib widget

game = SantoriniGame(5)

random_player = RandomPlayer(game).play
greedy_player = GreedySantoriniPlayer(game).play

args = dotdict({'numMCTSSims': 25, 'cpuct': 1.0})
mcts = MCTS(game, nnet, args)
alphago_player = lambda x: np.argmax(mcts.getActionProb(x, temp=0))

arena = Arena.Arena(alphago_player, greedy_player, game, display=game.display_3d)

%time oneWon, twoWon, draws = arena.playGames(5, verbose=False)
print("\AlphaGo won {} games, Greedy Player won {} games".format(oneWon, twoWon))

In [ ]:
# Checkpoints and best model (if found) will be saved in this folder
%ls temp